In [83]:
# ********** IMPORT LIBRARIES **************#


import numpy as np  # for handling multi-dimensional array operation
import pandas as pd  # for reading data from csv
#import statsmodels.api as sm  # for finding the p-value
from sklearn.preprocessing import MinMaxScaler  # for normalization
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle



In [86]:
# *********** Change all string feautres to numeric values ********

data=pd.read_csv('data/train.csv')
survival_map={1:1,0:-1}
gender_map={"male":1,"female":0}
data['Survived']=data["Survived"].map(survival_map)
data["Sex"]=data["Sex"].map(gender_map)

# data.drop(data.columns[[-1, 1]], axis=1, inplace=True)
data=data.drop(["Name",'SibSp',"Parch",'Ticket','Cabin',"Embarked"],axis=1)


In [87]:
#******** Removing Outliers using Inter Quarterile Range and replacing values with mean *********
data=data.loc[data['Age'].between(9,58),]
data=data.reset_index(drop=True,)

data=data.reset_index(drop=True,)
class_3_age=data.loc[data['Pclass']==3].Age.mean()
class_2_age=data.loc[data['Pclass']==2].Age.mean()
class_1_age=data.loc[data['Pclass']==1].Age.mean()
def remove_null_age(data):
    agee=data[0]
    pclasss=data[1]
    if pd.isnull(agee):
        if pclasss==1:
            return class_1_age
        elif pclasss==2:
            return class_2_age
        else:
            return  class_3_age

    return agee
data['Age']=data[["Age","Pclass"]].apply(remove_null_age,axis=1)
Y = data.loc[:, 'Survived']  # all rows of 'diagnosis'
X = data.iloc[:, 1:]
X=X.drop(["Survived"],axis=1)


In [ ]:
#*********** Normalizing the valeues *******************

X_normalized = MinMaxScaler().fit_transform(X.values)
X = pd.DataFrame(X_normalized)
 #for w.x+b we can directly implement --> x.w with x having a last column as 1's
# so
X.insert(loc=len(X.columns), column='intercept', value=1)


X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2, random_state=5)


In [ ]:
#************* Actual SVM Model *********************

def compute_cost(W,X,Y):
    N=X.shape(0)
    reg_strength = 20000
    distances=1-(Y*(np.dot(W,X)))
    print(distances[distances<0])
    distances[distances < 0] = 0
    hinge_loss = reg_strength * (np.sum(distances) / N)
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost

def calculate_grad_descent(x,y,W):
    # as implemented stochastic gd ...only singl vals of x,y are passed
    # so theyre basically ints...convert them to numpy array so
    # that this func also works wid basic vanilla gd also... where whole
    # features n outputs are passed in each iteration as (np arrays)...
    # actually theirs no other need to convert them to np arrays...
    # reg_strenght is the C val...regularisation term...tho here c=1/lambda
    # type is used...ie instead of c lambda is used...
    reg_strength = 20000
    if type(y) == np.int64 or type(y)==np.float64 :
        y = np.array([y])
        x = np.array([x])

    #distance :
    # if y(i)=1 --> wx-b>=1 ...so tht when we use max func 0 gets selected..
    # if y(i)= -1 --> wx-b<=-1 "" "" "" .. 1- (-1)(something less than -1)..
    # will result in 1- (something gr8er than 1) -1 n (-xyz) gets cancelled
    # this is the ideal situation we want... thats why we try to set W
    #  so as to get above result s and minimise cost...

    distance=1-(y*np.dot(x,W))
    derivative=np.zeros(len(W))

    for index,dist in enumerate(distance):
        if max(0,dist)==0:
            der=W
        else:
            der=W-(reg_strength*x*y)
        # print(np.shape(derivative))
        # print(np.shape(der))

        derivative+=np.squeeze(der)

    derivative=derivative/len(y)
    return derivative

def sgd(features,outputs):
    # features is our training set X and outputs is y
    #max number of tyms we want our sgd loop to run
    max_iters=10000
    # the w matrix of parameters ...these describe the boundary line
    # its values are manupilated so as to minimse the cost

    #weights is a 1-D array of dimensions(31,)...
    # its different from 2D array of 1 col i.e (31,1)
    # it has 31 rows and 1 column whereas former is just 1D 31 values
    # !!!!gives problems when multiplying tho looks same
    weights = np.zeros(features.shape[1])

    #iterate a fixed num of times for now.... later updating it wid stopag
    # criteria
    # TODO implement stoppage criteria st if cost isnt decreasing much
    #  or gd of consecutive values isnt much diff...stop
    for iters in range(1,max_iters):
        # shuffle values...cuz in sotchastic gd only 1 value enters the
        # formula of comute_gd...i.e tht funcn is called 430 tyms(no of rows)
        X,Y=shuffle(features,outputs)
        for i,x in enumerate(X):
            #gd for tht x value which entered in ccalce_grad_des func
            ascent=calculate_grad_descent(x,Y[i],weights)
            # 0.00001 is the learning rate "alpha" mostly known as
            weights=weights-(0.0001*ascent)

    return weights
print("training started...")

# X_train is a 455 row 31 column matrix with each column depecting a feature.
# sgd is stochastic gradient descent calculator.. in sgd func we find the S grad desc
# and traverse in opp directn of it.
W = sgd(X_train.to_numpy(), y_train.to_numpy())
print("training finished.")
print("weights are: {}".format(W))

In [ ]:
# ***** evaluating the model**********


y_test_predicted=np.array([])
for i,d in enumerate(y_test):
    yp=np.sign(np.dot(W,X_test.to_numpy()[i]))
    y_test_predicted=np.append(y_test_predicted,yp)

print("accuracy on test dataset: {}".format(accuracy_score(y_test.to_numpy(), y_test_predicted)))






